In [1]:
import os
import sys
sys.path.append(os.pardir)

import utils
from transformers import Mask2FormerImageProcessor, Mask2FormerForUniversalSegmentation
from torch.utils.data import DataLoader
import src.data.make_dataset as md

/opt/homebrew/anaconda3/envs/segmenting-subsurface-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = utils.get_config()
wandb = utils.init_wandb()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: louisreberga (association-rosia). Use `wandb login --relogin` to force relogin


In [3]:
processor = Mask2FormerImageProcessor.from_pretrained(
    wandb.config.model_id,
    do_rescale=False, 
    num_labels=1
)

model = Mask2FormerForUniversalSegmentation.from_pretrained(
    wandb.config.model_id,
    num_labels=1,
    ignore_mismatched_sizes=True
)

Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at facebook/mask2former-swin-large-coco-instance and are newly initialized because the shapes did not match:
- class_predictor.bias: found shape torch.Size([81]) in the checkpoint and torch.Size([2]) in the model instantiated
- class_predictor.weight: found shape torch.Size([81, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([81]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
set = 'train'
train_volumes = md.get_volumes(config, set=set)
args = {'config': config, 'processor': processor, 'set': set, 'volumes': train_volumes, 'dim': wandb.config.dim}
train_dataset = md.SegSubDataset(args)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=False, collate_fn=md.collate_fn)

In [8]:
batch = next(iter(train_dataloader))
item, inputs = batch

tensor(-0.0070) tensor(-0.0003)


ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-0.006995230447500944, -0.0003179650229867548] which cannot be converted to uint8.

In [6]:
inputs['pixel_values'].max()

tensor(0.2173)

In [7]:
inputs['pixel_values'].min()

tensor(-0.7479)